# Session 5 Exercises
These are sample answers for the in-class exercises in Session 5 of PHAS0030.  You should make sure that you can do these yourself ! The further work exercises will be in a separate notebook.

In [ ]:
# We always start with appropriate imports; note the use of the IPython magic
# command to set up Matplotlib within the notebook
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 3. Matrix approach to boundary-value problems

### Question 1

I think that this is the most compact way to create the matrix; note that the vectors that will form the off-diagonal entries are shorter than the full diagonal.

In [ ]:
# Set system length
N = 7
# Create arrays for diagonals
ondiag = -2*np.ones(N)
offdiag = np.ones(N-1)
# Assemble matrix and print
M = np.diag(ondiag) + np.diag(offdiag,k=1) +np.diag(offdiag,k=-1)
print(M)

### Question 2

Notice that the boundary condition vector **b** has only two entries, corresponding to the ends of the bar.  The sign of the vector comes from the algebra (in the notes, we have the first entry as $-c_{1,1}\theta_1$ with $c_{1,1}=1$ and the final entry as $-c_{N-2,N}\theta_N$ with $c_{N-2,N}=1$ because we have moved these to the right-hand side of the equation).

In [ ]:
# Set up boundary conditions
b = np.zeros(N)
# Note that these are negative - see equation in notes
b[0] = -500
b[N-1] = -300
# Solve for theta
Minv = np.linalg.inv(M)
theta = np.dot(Minv,b)
plt.plot(theta)
plt.xlabel('Index of point')
plt.ylabel('Temperature in K')
plt.title('Temperature in uniform bar')

This may look somewhat uninteresting (or even potentially wrong) but it is the correct steady-state solution: the temperature decreases evenly along the bar.  You could plot with points to check that you have the correct array displayed.

Notice that we are missing the end points in the solution: we included the boundary conditions in our calculation, but we did not calculate the ends.  Let's fix that.

In [ ]:
# Define a range large enough to include the ends
theta_full = np.zeros(N+2)
# Boundary conditions
theta_full[0] = 500
theta_full[N+1] = 300
# Copy solution
theta_full[1:N+1] = theta
# Plot
x = np.linspace(0,1,N+2)
plt.plot(x,theta_full)
plt.xlabel('Distance in m')
plt.ylabel('Temperature in K')
plt.title('Temperature in uniform bar')

## 5. Parabolic equations

### Question 1

I have made two functions here: the first iterates over the array, to make the algebra clear, while the second uses `np.roll` for efficiency.  Notice that in both cases we do not fix the boundary conditions, which must be done externally.

In [ ]:
def update_temperature(temper, zeta):
    """Perform explicit forward FD update for heat equation
    Inputs:
    temper  Array of temperature at present timestep
    zeta    Constant of proportionality
    
    Output:
    Array of temperature at next timestep
    """
    len_bar = np.size(temper)
    temper_next = np.zeros(len_bar)
    for i in range(1,len_bar-1): # Don't update end-points
        temper_next[i] = zeta*(temper[i+1] + temper[i-1]) + (1.0-2.0*zeta)*temper[i]
    return temper_next

In [ ]:
def update_temperature_alt(temper, zeta):
    """Perform forward FD update for heat equation using efficient approach
    Inputs:
    temper  Array of temperature at present timestep
    zeta    Constant of proportionality
    
    Output:
    Array of temperature at next timestep
    """
    # Second derivative (spatial) using np.roll
    temper_next = zeta*(np.roll(temper,-1) + np.roll(temper,1)) + (1.0-2.0*zeta)*temper
    # Fix end-points to zero (undefined here: should be defined externally)
    temper_next[0] = 0.0
    temper_next[-1] = 0.0
    return temper_next

### Question 2

We define $\theta_{0,i}$ as the initial condition for the problem via `temperature[0]=300` and `temperature[0,0]=500`. We also fix the temperature at the ends of the bar throughout the simulation via `temperature[:,0]=500` and `temperature[:,-1]=300`.  We then perform the integration in time to calculate values of temperature for all points on the bar for successive time steps.

In [ ]:
Nx = 7  # Seven points in spatial domain, ends will be fixed
Nt = 40 # Timesteps
temperature = np.zeros((Nt,Nx))
# Initial conditions (t=0)
temperature[0] = 300
temperature[0,0] = 500
# Boundary conditions (all times) though this may need updates
temperature[:,0] = 500 
temperature[:,-1] = 300 
zeta = 0.5
for n in range(1,Nt):
    temperature[n] = update_temperature_alt(temperature[n-1],zeta)
    # Set boundary conditions 
    temperature[n,0] = 500
    temperature[n,-1] = 300

### Question 3

Each line represents the temperature along the bar at a different timestep.  I've chosen to plot the lines every 5 steps (using `range(0,Nt,5)`) to simplify the plot.  (I have not calculated a timestep, dt: this would come from `zeta` and `dx` as well as the physical parameters of the system, but would make no significant difference to the plot.)

In [ ]:
x = np.linspace(0,1,Nx)
for n in range(0,Nt,5):
    plt.plot(x,temperature[n],label=f'step {n}')
plt.xlabel('x in m')
plt.ylabel('Temperature in K')
plt.title('Temperature distribution along bar for zeta=0.5')
plt.legend()

We see that the temperature propagates down the bar over time, as we might expect.  The long-time solution is exactly the linear decrease from 500K to 300K that we saw in the steady state solution (as we would expect - a useful check that we have met one condition).


In [ ]:
plt.imshow(temperature)
plt.colorbar(label='Temperature')
plt.xlabel('x index')
plt.ylabel('t index')

## Question 4

The critical value of zeta is a little larger than 0.5; note the scale of 1e9 in the top left hand corner of the graph (in a rather small font)

In [ ]:
# Initial conditions
temperature[0] = 300
# Boundary conditions
temperature[:,0] = 500 
temperature[:,N-1] = 300 
zeta = 0.7
for n in range(1,Nt):
    temperature[n] = update_temperature(temperature[n-1],zeta)
    temperature[n,0] = 500
    temperature[n,-1] = 300
for n in range(0,Nt):
    plt.plot(x,temperature[n])
plt.xlabel('x in m')
plt.ylabel('Temperature in K')
plt.title('Temperature distribution along bar for zeta=0.7')

## 6. Elliptic equations: iterative approaches

### Question 1

We include the basic Jacobi solver for completeness.

In [ ]:
def update_phi(phi, N):
    """Update NxN grid of phi using Jacobi method"""
    phiout = np.copy(phi)
    for i in range(1,N-1):
        for j in range(1,N-1):
            phiout[i,j] = 0.25*(phi[i-1,j] + phi[i+1,j] 
                              + phi[i,j-1] + phi[i,j+1])
    return phiout

In [ ]:
def update_phi_GS(phi, N):
    """Update NxN grid of phi using Gauss-Seidel method"""
    for i in range(1,N-1):
        for j in range(1,N-1):
            phi[i,j] = 0.25*(phi[i-1,j] + phi[i+1,j] + phi[i,j-1] + phi[i,j+1])
    return phi

In [ ]:
def update_phi_GS_SOR(phi, N, omega):
    """Update NxN grid of phi using SOR and Gauss-Seidel"""
    for i in range(1,N-1):
        for j in range(1,N-1):
            phi[i,j] = omega*0.25*(phi[i-1,j] + phi[i+1,j] + phi[i,j-1] + phi[i,j+1]) + (1 - omega)*phi[i,j]
    return phi

### Question 2

The heart of any relaxation calculation is a loop which continues until the change between successive solutions is smaller than a given tolerance.  As always, we should include a maximum number of iterations to prevent infinite loops (it is perfectly possible in this type of calculation to have a system which never reaches the tolerance).

In [ ]:
N = 15
phi = np.zeros((N,N))
# imshow expects the indices to be row, column
# So y=0 is the first row, or phi[0,:] which is counter-intuitive!
phi[0,:] = 4
phi[N-1,:] = 4
phi[:,0] = 3
phi[:,N-1] = 3
# Notice that these corners actually aren't used in the calculation
# but I define them for completeness
phi[0,0] = 3.5
phi[0,N-1] = 3.5
phi[N-1,0] =  3.5
phi[N-1,N-1] = 3.5

In [ ]:
# Tolerance and maximum number of iterations
tol = 1e-4
maxiter = 400
# Maximum change
delta = 1.0
iters = 1
while delta > tol and iters<maxiter:
    phiin = np.copy(phi)
    phi = update_phi_GS(phi,N)
    delta = np.max(np.abs(phiin - phi))
    iters += 1
print(f"Finished after {iters} iterations with maximum change {delta}")

### Question 3

I have chosen to use `plt.imshow` with interpolation, to give a smooth plot, but to add contours using `plt.contour` to make it more quantitative.  Note the parameter `levels` that can be passed to `plt.contour` and experiment to see the effect it has. (Notice also the value of the final level: I found that if I used 4.0 then the contour was not plotted; this must be rounding error.)  I also plot with `plt.contourf` below.

In [ ]:
plt.imshow(phi,origin='lower',interpolation='bicubic')
plt.colorbar(label='Potential (V)')
plt.contour(phi,colors='white',levels=[3.0,3.2,3.4,3.6,3.8,3.9999])
# This alternative gives the same set of contours as below
#plt.contour(phi,colors='white',levels=10)
plt.title('Map of potential')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
plt.contourf(phi,levels=10)
plt.axis('scaled')
plt.colorbar(label='Potential (V)')
plt.title('Potential with filled contours')
plt.xlabel('x')
plt.ylabel('y')

## 7. Supplementary: Elliptic equations: matrix approach

These solutions are provided to show you how this technique works; we mentioned it in class.  It is far from trivial to understand or implement, so do not worry if you didn't do this!

### Question 1

First, the indexing functions given in the notes

In [ ]:
def ij_to_index(i,j,N):
    """Convert i,j pair in (NxN) grid to index"""
    return i+j*N

In [ ]:
from math import floor
def index_to_ij(index,N):
    """Convert index to i,j pair in (NxN) grid"""
    j = floor(index/N)
    i = index - j*N
    return i,j

Now we calculate the Laplacian matrix and the boundary condition vector

In [ ]:
def laplace_matrix(N,bx,by):
    """Calculate matrix of Laplacian and boundary condition vector
    N is the total number of points in the grid
    """
    # Set up grid size
    Nx = int(N**0.5)
    # Build diagonal and sub-diagonal arrays
    ondiag = -4*np.ones(N)
    offdiag1 = np.ones(N-1)
    for i in range(1,Nx):
        offdiag1[i*Nx-1] = 0
    offdiag3 = np.ones(N-3)
    # Build matrix
    M = np.diag(ondiag) + np.diag(offdiag1,k=1) + np.diag(offdiag1,k=-1)
    M += np.diag(offdiag3,k=3) + np.diag(offdiag,k=-3)
    # Build RHS using boundary conditions
    b = np.zeros(N)
    for n in range(N):
        i,j = index_to_ij(n,Nx)
        if i==0 or i==Nx-1:
            b[n] -= bx
        if j==0 or j==Nx-1:
            b[n] -= by
    return M, b

In [ ]:
N=9
matMlap, blap = laplace_matrix(N,3,4)
print(matMlap)
print(blap)

### Question 2

Having built the matrix (which was complicated) the solution is almost trivial! The only problem is mapping the solution vector back onto the 2D grid.

In [ ]:
# We have a square array; define the size as Nx
Nx = int(np.sqrt(N))
solution = np.dot(np.linalg.inv(matMlap),blap)
potential = np.reshape(solution,(Nx,Nx))

I've decided here to create a larger array which includes the boundaries to make the plot more like the plots we saw above.  To do this I add the solution just created into the middle of the display array, and then set the boundaries.

In [ ]:
# Size of the array with boundary conditions
Nbc = Nx+2
# Create space for array
potential_bc = np.ones((Nbc,Nbc))
# Place solution in centre of array
potential_bc[1:Nbc-1,1:Nbc-1] = potential
# Set boundary conditions along edges
potential_bc[0] = 4
potential_bc[-1] = 4
potential_bc[:,0] = 3
potential_bc[:,-1] = 3
# Boundary conditions at the corners are not defined, but this makes a nicer plot
potential_bc[0,0] = 3.5
potential_bc[0,-1] = 3.5
potential_bc[-1,0] = 3.5
potential_bc[-1,-1] = 3.5
plt.imshow(potential_bc,interpolation='bicubic')
plt.colorbar(label='Potential(V)')

The interpolation makes the image look nicer (less blocky) but you must be careful with how it works.  We can also plot filled contours, or add contours to the image above.

In [ ]:
plt.contourf(potential_bc,levels=10)
plt.colorbar()
plt.axis('scaled')

Note that these are quite small grids; when you use the relaxation methods in the alternative approach, it is simple to extend to larger grids which are more realistic.  Notice also that we have not included the boundaries, which is why the plots look a little different to the previous results.